# Getting Started

IntelligentGraph follows the same pattern as RDFLib, since IntelligentGraph is derived from Graph. So everything that works for RDFLib works the same for IntelligentGraph.

## Installation

The latest release of RDFLib may be installed with Python's package management tool pip. Since IntelligentGraph depends on rdflib, version >7 of rdflib will be installed. However, since IntelligentGraph is still experimental it is loaded directly from GitHub

In [1]:
!pip install "git+https://github.com/peterjohnlawrence/IntelligentGraph.git"

  Cloning https://github.com/peterjohnlawrence/IntelligentGraph.git to /tmp/pip-req-build-r0a9embc
  Running command git clone --filter=blob:none --quiet https://github.com/peterjohnlawrence/IntelligentGraph.git /tmp/pip-req-build-r0a9embc
  Resolved https://github.com/peterjohnlawrence/IntelligentGraph.git to commit 20cfcb1b992e645b4448a88aa89756d28dd5f4a1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00
  Created wheel for IntelligentGraph: filename=IntelligentGraph-0.9.0-py3-none-any.whl size=9022 sha256=08b2ccd8261a81725ae4840c88da5ba475a1ef0d3336e5c0bcbcc43a3f304a29
  Stored in directory: /tmp/pip-ephem-wheel-cache-07m5pwsh/wheels/22/92/0f/992eb2e9d779860f2cfa4e85160e886f0bc904eba6ecd09d81
Successfully built IntelligentGraph


## Really simple start

The simplest example is to create an IntelkligentGraph with a single triple, whose value is a script that returns a literal. the value is return by assigning it to the predefined _result variable. We don't need a script to do this, but let's build up simply:

In [2]:
import intelligentgraph
from intelligentgraph import IntelligentConjunctiveGraph, IntelligentDataset,IntelligentGraph,SCRIPT
from rdflib import  Literal,   URIRef
from rdflib.namespace import FOAF
from datetime import date
g = IntelligentGraph()
ig = URIRef("http://inova8.com/ig")
g.add((ig, FOAF.birthday, Literal('''from datetime import date
_result =date(1951, 3, 8)''',datatype=SCRIPT.python)))

<Graph identifier=N3899f667e76841c3a7aeeec3448f954b (<class 'intelligentgraph.graph.IntelligentGraph'>)>

We can query this graph (with one triple!) as follows:

In [3]:
for triple in g.triples( (None , None, None)):
    print(triple)

(rdflib.term.URIRef('http://inova8.com/ig'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/birthday'), rdflib.term.Literal('1951-03-08', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))


Note that the object's script value has been replaced with a literal containing the xsd:date value that was returned from the script when it was evaluated.

## Let's get a bit more real
If we wanted to know a person's age, we could of course query this graph, and as part of the SPARQL query, or Python code handling the returned values, calculate the age in years. For example

In [4]:
s = URIRef("http://inova8.com/ig")

for triple in g.triples( (s , FOAF.birthday, None)):
    age= int((date.today()-triple[2].toPython()).days/365.25)
    print (age)

72


However, if we were to share the graph with someone, they would not have access to this code unless we shared that as well. Why not incorporate the age calculation within the graph? This is analogous to a spreadsheet in which some cell values are literals whilst others are calculations based on other cell values. Let's add a calculation script for the FOAF:age as follows. Note that the script will be initialized with values as follows:

- The value 's' is supplied to the script as the subject of the triple with which the script is associated as the object.
- The predicate of the same triples is supplied a variable 'p'.
- If we are using 'quads' then the context is provided as variable 'ctx'.
- Finally, the graph within which the triple is defined is provided as variable 'g'

In [5]:
g.add((ig, FOAF.age, Literal('''
from rdflib.namespace import FOAF
from datetime import date
for triple in g.triples( (s , FOAF.birthday, None)):
    age= int((date.today()-triple[2].toPython()).days/365.25)
    _result = age''',datatype=SCRIPT.python)))

<Graph identifier=N3899f667e76841c3a7aeeec3448f954b (<class 'intelligentgraph.graph.IntelligentGraph'>)>

We can view the calculated graph using

In [6]:
print(g.serialize(format="n3"))

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://inova8.com/ig> foaf:age 72 ;
    foaf:birthday "1951-03-08"^^xsd:date .




## Fetching External Data

There are many occasions when we want to merge a graph with external information. Usually, this is done by running some code that retrieves the triples from the external data source, such as an IoT server. This data then has to be merged with the underlying graph.
IntelligentGraph offers a better alternative: add an agent *within* the graph that pulls this external data just-in-time, instead of just-in-case.

Let's start by simulating an external service that returns the FOAF:knows triples about a particular individual. In reality, we will use an external service (https://random-word-api.herokuapp.com) that generates random names.

In [7]:
s = URIRef("http://inova8.com/ig")
import requests
def getKnows(individual):
  response = requests.get("https://random-word-api.herokuapp.com/word?lang=en&number=5")
  for word in response.json():
    yield (individual, FOAF.knows, URIRef("http://inova8.com/"+word))
for triple in getKnows(s):
  print(triple)

(rdflib.term.URIRef('http://inova8.com/ig'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('http://inova8.com/yeasayers'))
(rdflib.term.URIRef('http://inova8.com/ig'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('http://inova8.com/massifs'))
(rdflib.term.URIRef('http://inova8.com/ig'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('http://inova8.com/tittivate'))
(rdflib.term.URIRef('http://inova8.com/ig'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('http://inova8.com/reechy'))
(rdflib.term.URIRef('http://inova8.com/ig'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('http://inova8.com/skreighing'))


Instead of merging these two graphs, let's add an agent script that returns these results. we use the 'yield' pattern so that a script need not fetch all results. Instead, yield allows each value to be fetched on request:

In [8]:
g.add((ig, FOAF.knows, Literal('''
import requests
def getKnows(individual):
  response = requests.get("https://random-word-api.herokuapp.com/word?lang=en&number=5")
  for word in response.json():
    yield (individual, FOAF.knows, URIRef("http://inova8.com/"+word))
_result = getKnows(s)''',datatype=SCRIPT.python)))

<Graph identifier=N3899f667e76841c3a7aeeec3448f954b (<class 'intelligentgraph.graph.IntelligentGraph'>)>

We can again view the entire graph simply by serializing it as follows:

In [9]:
print(g.serialize(format="n3"))

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://inova8.com/ig> foaf:age 72 ;
    foaf:birthday "1951-03-08"^^xsd:date ;
    foaf:knows <http://inova8.com/administrant>,
        <http://inova8.com/geste>,
        <http://inova8.com/giardiases>,
        <http://inova8.com/rousingly>,
        <http://inova8.com/strenuousness> .




## Summary

In this example, the asserted graph only contains three statements, each of which  has a script for an object value:

```python
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
<http://inova8.com/ig>
    foaf:birthday """from datetime import date
                _result =date(1951, 3, 8)"""^^<http://inova8.com/script/python> ;
    foaf:age """
                from rdflib.namespace import FOAF
                from datetime import date
                for triple in g.triples( (s , FOAF.birthday, None)):
                age= int((date.today()-triple[2].toPython()).days/365.25)
                _result = age"""^^<http://inova8.com/script/python> ;

    foaf:knows """
                import requests
                def getKnows(individual):
                    response = requests.get("https://random-word-api.herokuapp.com/word?lang=en&number=5")
                    for word in response.json():
                        yield (individual, FOAF.knows, URIRef("http://inova8.com/"+word))
                _result = getKnows(s)"""^^<http://inova8.com/script/python> .
  ```

When queried as an IntelligentGraph, it will appear that the graph contains the following:
```python
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://inova8.com/ig> foaf:age 72 ;
    foaf:birthday "1951-03-08"^^xsd:date ;
    foaf:knows <http://inova8.com/anthodium>,
        <http://inova8.com/batts>,
        <http://inova8.com/exodermises>,
        <http://inova8.com/fascicled>,
        <http://inova8.com/hangers> .
```